In [30]:
%load_ext autoreload
%autoreload 2

import misc
import config
import numpy as np

layers_indexes = config.get_layers_indexes(4)
config = config.Config()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
genome = np.array([[1,0,0,0,0,1,0,0,0,0,9,9,9,9,0]])
module_genome = misc.module_convert(genome[0], layers_indexes)
module_graph = misc.get_graph(module_genome)


In [32]:
print(module_genome)
print(module_graph)

[array([1]), array([0, 0]), array([0, 0, 1])]
{1: [], 2: [1], 3: [], 4: [3], 5: [2, 4]}


In [35]:
print(misc.get_graph(misc.module_convert(misc.RemoveDisconnectedLayers(genome, config).return_new_X()[0], layers_indexes)))


{1: [], 2: [1], 3: [], 4: [], 5: [2]}
